In [1]:
import os
os.environ["JAVA_HOME"] = "/Users/sribalac/Library/Java/JavaVirtualMachines/liberica-1.8.0_322"
os.environ["SPARK_HOME"] = "/Users/sribalac/Documents/Data Engg Tutorial/Assignment 1/spark-3.0.3-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

DATASET_ROOT = "/Users/sribalac/Documents/Data Engg Tutorial/Anime-Project-BE/resources/anime-dataset/"

spark = SparkSession.builder \
    .master("local[5]") \
    .appName("Colab") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

22/12/09 17:19:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/09 17:19:10 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [2]:
def process_batch(df, epoch_id):
    df.sort(desc("count")).limit(15).toPandas().to_json("/Users/sribalac/Documents/Data Engg Tutorial/Anime-Project-BE/resources/output.json")

In [3]:
data_schema = StructType() \
    .add("time", "timestamp") \
    .add("anime_id", "string") \
    .add("watchDuration", "integer")

anime_df = spark.readStream.option("maxFilesPerTrigger", 1).schema(data_schema) \
    .csv("/Users/sribalac/Documents/Data Engg Tutorial/Anime-Project-BE/resources/resultThree/*.csv")

windowed = anime_df.withWatermark("time","10 minutes").groupBy("anime_id", window("time","10 minutes","10 minutes")).count()
query = windowed.writeStream.foreachBatch(process_batch).outputMode("Update").start().awaitTermination()
# This will run until terminated manually

22/12/09 17:19:21 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/t4/crxs1d5n4rg_08ctnzw10m3w0000gn/T/temporary-b0b11ebd-c665-43a3-b3cb-9eca7716ab8c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/12/09 17:19:28 WARN FileStreamSource: Listed 5000 file(s) in 6572 ms         
22/12/09 17:19:44 WARN FileStreamSource: Listed 5000 file(s) in 5718 ms         
22/12/09 17:19:53 WARN FileStreamSource: Listed 5000 file(s) in 5652 ms         
22/12/09 17:20:02 WARN FileStreamSource: Listed 5000 file(s) in 5309 ms         
22/12/09 17:20:12 WARN FileStreamSource: Listed 5000 file(s) in 6268 ms         
22/12/09 17:20:21 WARN FileStreamSource: Listed 5000 file(s) in 5372 ms         
22/12/09 17:20:31 WARN FileStreamSource: Listed 5000 file(s) in 5665 ms     

KeyboardInterrupt: 